In [10]:
import torch
tv = torch.__version__
cv = torch.version.cuda
# print(cv)  # change this to the following format...
cv = "cu101"
# print(tv)
link = "https://pytorch-geometric.com/whl/torch-" + tv + ".html"
!pip install torch-scatter==latest+{cv} -f {link}
!pip install torch-sparse==latest+{cv} -f {link}
!pip install torch-cluster==latest+{cv} -f {link}
!pip install torch-spline-conv==latest+{cv} -f {link}
!pip install torch-geometric
!pip install networkx 
!pip install vincenty  # used for distance calculation between cities

  Created wheel for vincenty: filename=vincenty-0.1.4-py3-none-any.whl size=3084 sha256=fbdddd2d20cd539d4fcd17248613d4883321629ca0baf9d53c124215948a27b0
  Stored in directory: c:\users\asaf karnieli\appdata\local\pip\cache\wheels\2f\4d\df\6e2ce31c63c93508b38ed9098c1483e7616b55a6ddabcd2f5b
Successfully built vincenty


In [11]:
# get data from here - https://www.kaggle.com/berkeleyearth/climate-change-earth-surface-temperature-data?select=GlobalLandTemperaturesByCity.csv

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx
from vincenty import vincenty

# read data, take only data for USA at the most recent date (day)
df = pd.read_csv('../GlobalLandTemperaturesByCity.csv')
df = df[df['Country']=="United States"]
df = df[df['dt']=="2013-09-01"]
df = df.fillna(0)



In [12]:
df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
49879,2013-09-01,25.791,1.180,Abilene,United States,32.95N,100.53W
140304,2013-09-01,17.799,1.093,Akron,United States,40.99N,80.95W
170399,2013-09-01,19.207,0.866,Albuquerque,United States,34.56N,107.03W
190766,2013-09-01,19.643,1.050,Alexandria,United States,39.38N,76.99W
205489,2013-09-01,17.408,1.048,Allentown,United States,40.99N,74.56W


In [ ]:
# create the graph - 
graph= networkx.Graph()
# todo - add vertices - one for each city



for idx1, itm1 in enumerate(data):
    for idx2, itm2 in enumerate(data):
            pos1 = (itm1["lng"], itm1["lat"])
            pos2 = (itm2["lng"], itm2["lat"])
            distance = vincenty(pos1, pos2).meters #geopy distance
            # print(idx1, idx2, distance)
            graph.add_edge(idx1, idx2, weight=distance)